In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!ls /content/drive/MyDrive

Mounted at /content/drive
 20191216_122821.mp4		  gptneo-finetuned-qa
 20200101_074036.mp4		  math
 20200324_211649.mp4		 'Screenshot_20220307-174728_().jpg'
 Classroom			  대마도여행250224.gmap
'Colab Notebooks'		  여수여행_241216.gmap
 Colab_Notebooks		 '역사 보고서.show'
'Gantt Chart_07의 사본.gslides'   오키나와여행_241230.gmap
 gptneo-1.3B-finetuned		  통지서.html
 gptneo-1.3B-masked		  홍콩여행_240121.gmap


In [2]:
# 1. 필요한 패키지 설치
!pip install fastapi pyngrok uvicorn fsspec==2025.3.0 transformers datasets accelerate nest_asyncio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from fastapi import FastAPI
from pydantic import BaseModel
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# 모델 로딩
model_path = "/content/drive/MyDrive/math"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")
tokenizer.pad_token = tokenizer.eos_token

In [7]:
# 6. FastAPI app 설정
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class ProblemRequest(BaseModel):
    topic: str
    max_new_tokens: int = 100
    temperature: float = 0.8

class ProblemResponse(BaseModel):
    response: str

@app.post("/generate", response_model=ProblemResponse)
def generate_problems(req: ProblemRequest):
    # 명확한 프롬프트 설계
    prompt = f"주제: {req.topic}\n\n문제:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        inputs["input_ids"],
        max_new_tokens=req.max_new_tokens,
        do_sample=True,
        temperature=req.temperature,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return ProblemResponse(
        response=json.dumps(generated_text.strip(), ensure_ascii=False, indent=2)
    )

In [ ]:
# 7. Run ngrok + uvicorn server (Colab only)
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import json

ngrok.set_auth_token("2v1Fi5CEzLumREBpheNMIIepRlM_7uLFbq5PGe81hmEZiAe9K")
ngrok.kill()

public_url = ngrok.connect(3000)
print("🔗 Public URL:", public_url.public_url)

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=3000)

🔗 Public URL: https://7dcd-35-190-156-234.ngrok-free.app


INFO:     Started server process [1094]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:3000 (Press CTRL+C to quit)


INFO:     155.230.85.158:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     155.230.85.158:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     155.230.85.158:0 - "POST /generate HTTP/1.1" 200 OK


In [ ]:
from google.colab import files
import json

uploaded = files.upload()

with open("math.json", "r", encoding="utf-8") as f:
  raw_data = json.load(f)

Saving math.json to math.json


In [ ]:
!pip install textattack==0.3.5
!pip install transformers
!pip install torch

  Using cached textattack-0.3.5-py3-none-any.whl.metadata (37 kB)
  Using cached datasets-1.15.0-py3-none-any.whl.metadata (19 kB)
  Using cached pycld2-0.42-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.5 kB)
  Using cached click-8.0.4-py3-none-any.whl.metadata (3.2 kB)
  Using cached huggingface_hub-0.0.19-py3-none-any.whl.metadata (6.3 kB)
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached transformers-4.51.2-py3-none-any.whl.metadata (38 kB)
  Using cached transformers-4.51.1-py3-none-any.whl.metadata (38 kB)
  Using cached transformers-4.51.0-py3-none-any.whl.metadata (38 kB)
  Using cached transformers-4.50.3-py3-none-any.whl.metadata (39 kB)
  Using cached transformers-4.50.2-py3-none-any.whl.metadata (39 kB)
  Using cached transformers-4.50.1-py3-none-any.whl.metadata (39 kB)
  Using cached transformers-4.50.0-py3-none-any.whl.metadata (39 kB

In [ ]:
import textattack

In [ ]:
import textattack.augmentation

for name in dir(textattack.augmentation):
    if not name.startswith("__"):
        print(name)

Augmenter
BackTranslationAugmenter
CLAREAugmenter
CharSwapAugmenter
CheckListAugmenter
DeletionAugmenter
EasyDataAugmenter
EmbeddingAugmenter
WordNetAugmenter
augmenter
recipes


In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
# 필요한 라이브러리 설치
!pip install transformers sentencepiece

# 라이브러리 import
import random
import requests
import time
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# 1. Papago API 설정
client_id = '64yuz72rjr'
client_secret = 'mpxNeA4kw4gIEe2TgfRDfnWSzeWBJ1Sz0FfLkUDW'

def papago_translate(text, source_lang, target_lang):
    url = "https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret
    }
    data = {
        "source": source_lang,
        "target": target_lang,
        "text": text
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()['message']['result']['translatedText']
    else:
        print(f"번역 오류 발생: {response.status_code}, 응답: {response.text}")
        return None

# 2. HuggingFace Paraphraser 로딩
model_name = "ramsrigouthamg/t5_paraphraser"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

paraphraser = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

def generate_paraphrases(text, num_return_sequences=3):
    input_text = f"paraphrase: {text} </s>"
    outputs = paraphraser(
        input_text,
        max_length=100,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=100,
        top_p=0.92,
        temperature=0.8
    )
    return [output['generated_text'] for output in outputs]

# 3. Synonym 치환 (가벼운 단어 변형)
synonym_dict = {
    # 상식 관련
    "국가": ["나라","국가","국민국"],
    "수도": ["중심 도시","수도 도시","행정 중심지"],
    "발명": ["개발","창안","고안"],
    "역사": ["과거","연혁","기록"],
    "과학": ["자연 과학","학문","이론 체계"],
    "환경": ["자연환경","생태계","주변 환경"],
    "경제": ["재정","금융","산업"],

    # 수학 관련
    "합": ["덧셈 결과","총합","전체 합계"],
    "곱": ["곱셈 결과","곱셈값","곱한 값"],
    "차이": ["뺄셈 결과","차","차분"],
    "비율": ["비","비례 관계","비율 값"],
    "각도": ["회전량","각의 크기","각도값"],
    "수열": ["수의 나열","연속된 수","순서열"],
    "방정식": ["수식","등식","연립식"],

    # 논리 관련
    "참": ["진리","사실","진실"],
    "거짓": ["허위","잘못","오류"],
    "전제": ["가정","조건","기본 가정"],
    "결론": ["결과","추론 결과","최종 판단"],
    "증명": ["논증","입증","확인"],
    "명제": ["주장","문장","서술"],
    "추론": ["유추","논리적 도출","이끌어냄"]
}

def synonym_replace(korean_text):
    words = korean_text.split()
    new_words = []
    for word in words:
        replaced = False
        for key, synonyms in synonym_dict.items():
            if key in word:
                if random.random() < 0.5:  # 50% 확률로 교체
                    word = word.replace(key, random.choice(synonyms))
                    replaced = True
                    break
        new_words.append(word)
    return ' '.join(new_words)

# 4. Style Transfer (공손체 ↔ 구어체 변형)
def style_transfer(text, apply_prob=0.6):
    if random.random() > apply_prob:
        return text

    endings = {
        "했습니다.": ["해요.", "했어요."],
        "했습니다": ["해요", "했어요"],
        "한다.": ["해.", "했어."],
        "합니다.": ["해요.", "했어요."],
        "합니다": ["해요", "했어요"],
        "있습니다.": ["있어요.", "있어."],
        "있습니다": ["있어요", "있어"]
    }

    for formal, casual_list in endings.items():
        if formal in text:
            text = text.replace(formal, random.choice(casual_list))
    return text

# 5. 전체 프로세스 함수
def full_back_translate_with_paraphrase_style(korean_text, num_augments=3):
    #print(f"[원본 문장]: {korean_text}")

    # Step 1: 한국어 -> 영어
    en_text = papago_translate(korean_text, source_lang='ko', target_lang='en')
    time.sleep(0.2)

    # Step 2: 영어 Paraphrasing
    paraphrases = generate_paraphrases(en_text, num_return_sequences=num_augments)
    time.sleep(0.2)

    results = []
    for p in paraphrases:
        # Step 3: 영어 -> 한국어
        ko_text = papago_translate(p, source_lang='en', target_lang='ko')
        time.sleep(0.2)

        # Step 4: Synonym 치환
        ko_text_synonym = synonym_replace(ko_text)

        # Step 5: Style 변형
        final_text = style_transfer(ko_text_synonym)

        results.append(final_text)

   # print("\n[최종 변형된 문장들]:")
   # for idx, text in enumerate(results, 1):
   #     print(f"  {idx}: {text}")

    return results

# 6. 테스트 실행
test_sentence = "사람들이 오직 아들만 원하는 나라에서 모든 가족들은 아들을 낳을 때까지 계속 아이를 낳습니다."
#augmented_sentences = full_back_translate_with_paraphrase_style(test_sentence, num_augments=5)
#augmented_sentences
for item in raw_data:
    question = item['Question']
    answer = item['Answer']

    augmented_sentences = full_back_translate_with_paraphrase_style(question, 4)

    print("Original Question: ", question)
    for i in range(4):
      print(f"Augmented Question {i+1}: ", augmented_sentences[i])
    print("Answer: ", answer, "\n")

Device set to use cpu


Original Question:  사람들이 오직 아들만 원하는 나라에서 모든 가족들이 아들을 낳을 때까지 계속 아이를 낳는다. 만일 그들이 딸을 갖는다면 그들은 또 다른 아이를 가지게 된다. 만일 그들이 아들을 낳는다면 더 이상 아이를 갖지 않는다. 이 나라에서 아들과 딸의 성비는?
Augmented Question 1:  사람들이 아들만 원하는 나라에서는 모든 가정이 아들을 낳을 때까지 계속 아이를 낳습니다. 그들에게 딸이 있다면, 또 다른 아이가 있어. 아들이 있으면 더 이상 자녀가 없습니다. 이 나라에서 아들과 딸의 성비는 어떻게 되나요?
Augmented Question 2:  사람들이 아들만 원하는 나라에서는 모든 가정이 딸을 낳을 때까지 자녀를 계속 낳습니다. 그들에게 아들이 있다면, 또 다른 아이가 있습니다. 아들이 있으면 더 이상 자녀가 없습니다. 이 나라에서 아들과 딸의 성비는 어떻게 되나요?
Augmented Question 3:  사람들이 아들만 원하는 나라에서는 모든 가정이 아들을 낳을 때까지 계속 아이를 낳습니다. 그들에게 딸이 있다면, 또 다른 아이가 있어. 아들이 있으면 더 이상 자녀가 없습니다. 이 나라에서 아들과 딸의 성비는 어떻게 되나요?
Augmented Question 4:  사람들이 아들만 원하는 나라에서는 모든 가정이 딸을 낳을 때까지 자녀를 계속 낳습니다. 그들에게 딸이 있다면, 또 다른 아이가 있습니다. 아들이 있으면 더 이상 자녀가 없습니다. 이 나라에서 아들과 딸의 성비는 어떻게 되나요?
Answer:  1:1 

Original Question:  하루에 시계의 분침과 시침은 몇 번 겹치는가?
Augmented Question 1:  시계의 시침과 분침은 하루에 몇 번 겹치나요?
Augmented Question 2:  시계의 분침과 시침은 하루에 몇 번 겹치나요?
Augmented Question 3:  시계의 분침과 시침은 하루에 몇 번 겹치나요?
Augmented Question 4:  시계의 시침과 분침

KeyboardInterrupt: 